In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset, TepKasperskyDataset, SwatItrustDataset
from utils.custom_plots import plot_stacked
from utils.metrics import time_span_metrics
from utils.watchmen import LimitWatchman, LimitPcaWatchman, SpePcaWatchman, IsolatingWatchman, LinearPredictWatchman

In [4]:
SEED = 1729

# Preparing datasets

In [5]:
datasets = {
    0: GhlKasperskyDataset(),
    1: TepHarvardDataset(),
    2: TepKasperskyDataset(),
    3: SwatItrustDataset(),
}

In [6]:
shake_kwargs = {
    'random_state': SEED,
    'valid_test_ratio': 0.3,
}

# Preparing watchmen

In [7]:
watchmen = {
    0: {  # GhlKasperskyDataset
        0: LimitWatchman(),
        1: LimitPcaWatchman(n_components=3),
        2: SpePcaWatchman(n_components=3),
        3: IsolatingWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
    },
    1: {  # TepHarvardDataset
        0: LimitWatchman(ewma='3 min'),
        1: LimitPcaWatchman(n_components=12),
        2: SpePcaWatchman(n_components=12),
        3: IsolatingWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
    },
    2: {  # TepKasperskyDataset
        0: LimitWatchman(),
        1: LimitPcaWatchman(n_components=3),
        2: SpePcaWatchman(n_components=3),
        3: IsolatingWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
    },
    3: {  # SwatItrustDataset
        0: LimitWatchman(),
        1: LimitPcaWatchman(n_components=3),
        2: SpePcaWatchman(n_components=3),
        3: IsolatingWatchman(random_state=SEED),
        4: LinearPredictWatchman(random_state=SEED),
    },
}

# Learning

## Prefit

In [9]:
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)
    for data, _, _ in tqdm(datasets[d].train_generator(), desc=f'Prefit on train {d}'):
        for w in watchmen[d]:
            watchmen[d][w].prefit(data)

Prefit on train 0: 0it [00:00, ?it/s]

Prefit on train 1: 0it [00:00, ?it/s]

Prefit on train 2: 0it [00:00, ?it/s]

Prefit on train 3: 0it [00:00, ?it/s]

## Fit

In [10]:
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)
    for data, _, _ in tqdm(datasets[d].train_generator(), desc=f'Fit on train {d}'):
        for w in watchmen[d]:
            watchmen[d][w].partial_fit(data)

Fit on train 0: 0it [00:00, ?it/s]

Fit on train 1: 0it [00:00, ?it/s]

Fit on train 2: 0it [00:00, ?it/s]

Fit on train 3: 0it [00:00, ?it/s]

# Examine

## Throw stones

In [11]:
stones = dict()
for d in datasets:
    stones[d] = dict()
    for w in watchmen[d]:
        stones[d][w] = list()

In [12]:
for d in datasets:
    datasets[d].shake_not_stir(**shake_kwargs)
    for data, faults, info in tqdm(datasets[d].valid_generator(), desc=f'Detect on validation {d}'):
        for w in watchmen[d]:
            detect = watchmen[d][w].predict(data)
            stones[d][w].append(pd.concat([faults, detect], axis=1))
            stones[d][w][-1].index.name = info

Detect on validation 0: 0it [00:00, ?it/s]

Detect on validation 1: 0it [00:00, ?it/s]

Detect on validation 2: 0it [00:00, ?it/s]

KeyboardInterrupt: 

## Individual results

In [16]:
metrics = ('precision', 'recall', 'f1_score')
indi_results = dict()
for d in datasets:
    indi_results[str(datasets[d])] = pd.DataFrame(columns=metrics)
    for w in watchmen[d]:
        exam_paper = pd.DataFrame(columns=metrics)
        for st in stones[d][w]:
            exam_paper.loc[st.index.name, metrics] = time_span_metrics(st.iloc[:, 0], st.iloc[:, 1:])
        indi_results[str(datasets[d])].loc[str(watchmen[d][w]), metrics] = exam_paper.mean().values

In [17]:
for d in indi_results:
    print(d)
    display(indi_results[d])

GhlKasperskyDataset(E:\Datasets\GHL)


,precision,recall,f1_score
LimitWatchman(ewma=None),0.207358,0.638095,0.281676
LimitPcaWatchman(n_components=3),0.284864,0.328571,0.298718
SpePcaWatchman(n_components=3),0.0,0.0,0.0
IsolatingWatchman(n_trees=17),0.017045,1.0,0.033429
LinearPredictWatchman(n_features=12),0.014094,1.0,0.02773


TepHarvardDataset(E:\Datasets\TEP\dataverse)


,precision,recall,f1_score
LimitWatchman(ewma=3 min),1.0,0.643558,0.68672
LimitPcaWatchman(n_components=12),1.0,0.510827,0.553397
SpePcaWatchman(n_components=12),1.0,0.574329,0.611637
IsolatingWatchman(n_trees=500),0.845354,0.4658,0.530065
LinearPredictWatchman(n_features=52),0.9996,0.750827,0.788164


TepKasperskyDataset(E:\Datasets\TEP\kaspersky)


,precision,recall,f1_score
LimitWatchman(ewma=None),0.547695,0.397086,0.33061
LimitPcaWatchman(n_components=3),0.75,0.2,0.2
SpePcaWatchman(n_components=3),0.977778,0.210353,0.219549
IsolatingWatchman(n_trees=400),0.06954,1.0,0.110505
LinearPredictWatchman(n_features=53),0.267029,0.372862,0.184011


SwatItrustDataset(E:\Datasets\SWaT\dataset12)


,precision,recall,f1_score
LimitWatchman(ewma=None),NaN,NaN,NaN
LimitPcaWatchman(n_components=3),NaN,NaN,NaN
SpePcaWatchman(n_components=3),NaN,NaN,NaN
IsolatingWatchman(n_trees=6),NaN,NaN,NaN
LinearPredictWatchman(n_features=51),NaN,NaN,NaN


## Ensembling results

### At least one

In [26]:
metrics = ('precision', 'recall', 'f1_score')
union_results = dict()
for d in datasets:
    union_results[str(datasets[d])] = pd.DataFrame(columns=metrics)
    exam_paper = pd.DataFrame(columns=metrics)
    for i_st, st in enumerate(stones[d][0]):
        detect_union = pd.concat([stones[d][w][i_st].iloc[:, 1:] for w in watchmen[d]], axis=1)
        exam_paper.loc[st.index.name, metrics] = time_span_metrics(st.iloc[:, 0], detect_union)
    union_results[str(datasets[d])].loc['at_least_one', metrics] = exam_paper.mean().values

IndexError: list index out of range

In [27]:
for d in union_results:
    print(d)
    display(union_results[d])

GhlKasperskyDataset(E:\Datasets\GHL)


,precision,recall,f1_score
at_least_one,0.014094,1.0,0.02773


TepHarvardDataset(E:\Datasets\TEP\dataverse)


,precision,recall,f1_score
at_least_one,0.919894,0.789587,0.821137


TepKasperskyDataset(E:\Datasets\TEP\kaspersky)


,precision,recall,f1_score


### Two or more

### Weighted